# _Scala Fundamentals_
# 4. Miscellany

_This module was originally developed by Marcus Henry, Jr. It has been updated and expanded a bit._

## Basic uses for packages and types in Scala

### Packages

- How to define a package
- How to access members of a package
- How to pull members of a package into scope

### Types

- What expressions in Scala are objects and what expressions are not objects
- The Scala type hierarchy

## Packages in the wild

These aren't in code cells, because there's no real way to define
packages in the Ammonite REPL. (In the Scala REPL, you can do it,
but you have to use the special `:paste` command.)

### Create a package with some contents

Formal syntax

```scala
package org.example.outer {
  package inner {
    object Something
    object Something1
    object Something2
    object Something3
    object Something4
    object Something5
  }
}
```

There are some esoteric reasons to use that syntax. But we usually use a more compact syntax:

```scala
package org.example.outer.inner

object Something
object Something1
object Something2
object Something3
object Something4
object Something5
```

Let's access some values in an existing package, [`scala.math`](https://www.scala-lang.org/files/archive/api/2.13.0/scala/math/index.html).

In [ ]:
scala.math.max(10, 20)
scala.math.pow(3, 3)
scala.math.cos(10.0)
scala.math.abs(-100)
scala.math.sqrt(2)

## Imports

Obviously, it's tedious to type the full package name every time.

### Importing the package

We can shorten it by importing `scala.math`.

In [ ]:
import scala.math
math.max(10, 20)
math.pow(3, 3)
math.cos(10.0)
math.abs(-100)
math.sqrt(2)

### Importing specific members

We can also import specific members — functions, values, and subpackages, if there are any.

In [ ]:
import scala.math.max // import just one member
import scala.math.{pow, cos, sqrt} // import multiple specific members

max(10, 20)
pow(2, 5)
cos(5)
sqrt(3)

### Importing all members (wildcard import)

More underscore abuse.

In [ ]:
import scala.math._

sin(3)
tan(5)
log10(100)

### Import with rename

We can import members and rename them, so they don't conflict with something else.

In [ ]:
def max(nums: Int*): Int = nums.sorted.reverse.head

max(1, 30, 100, 3)

In [ ]:
import scala.math.{abs, max => mathmax}

max(1, 30, 100, 3)
abs(mathmax(-1, -30))

### Import everything but...

Let's import everything from a package _except_ for one item.

(Here's that underscore again.)

In [ ]:
import java.time.{Clock => _, _}

LocalTime.now
ZonedDateTime.now(ZoneId.of("UTC"))

// This would fail, because we excluded Clock from the import.
//Clock.systemDefaultZone


## Recap

- Packages encapsulate data and functions helping to add some organization to the code.
- Imports break this encalsulation allowing one package to access members of another.
    - Imports can be compactified into single liners if necessary
- The type system will not allow identical symbols to be used at the same time.
    - Beware of name clashes (especially when importing with `_`)
- We can mix and match symbols freely, as long as their names in scope are different

## Imports don't have to be at the top of the file

Imports can occur inside any block, which allows you to limit their scope.

In [ ]:
def sortByAbs(nums: Int*): Seq[Int] = {
  import scala.math.abs
  
  // abs is visible inside the function.
  nums.map(abs).sorted
}

// abs is not visible here.

sortByAbs(-1, 2, 10, -3, -100)

val now = {
  import java.time.{ZonedDateTime, ZoneId}
  ZonedDateTime.now(ZoneId.of("UTC"))
}

// ZonedDateTime and ZoneId aren't visible here.

## You can import from objects and classes

Scala also permits importing from `object` singletons and instantiated objects.

### Import from a singleton `object`

In [ ]:
object MyMathStuff {
  def manyMax(nums: Int*): Int = nums.sorted.reverse.head
  def harmonicMean(item: Int, items: Int*): Double = {
    if (items.toList.isEmpty)
      item.toDouble
    else {
      val allItems = item +: items
      allItems.length / allItems.foldLeft(0.0)((a, b) => a + (1.0 / b.toDouble))
    }
  }
}

In [ ]:
import MyMathStuff._

manyMax(-1, 10, 30, 100)
harmonicMean(1, 2, 15, 10, 20, 3)

### Import from an instantiated object

This is a contrived example, but you _will_ see instances of this practice in the wild.

In [ ]:
val nameAges = Map(
  "Abby"      -> 14,
  "Jamal"     -> 12,
  "Kumar"     -> 20,
  "Genevieve" -> 31
)
import nameAges.get
get("Abby")
get("Joe")

### Imports summary

- Imports in Scala are more powerful than imports in Java.
- Imports can be multiple on the same line.
- Imports can rename classes and members just for the enclosing scope.
- Imports can be placed anywhere in the file, even inside of classes.
- You can import from objects, not just packages.

## Let's talk about types

- Everything in Scala is an expression.
- Expressions have a type.
- Types tell the compiler how to interpret your code.
- _Everything_ in Scala is an object, at least at compile time (in contrast to Java).

In [ ]:
1.toString
1L.toString
'c'.toUpper

### Type hierarchy

But, even though Scala wants to present the world as entirely composed of objects, 
the JVM doesn't work that way. The JVM:

- has primitives (`int`, `char`, `long`, etc.)
- has a primitive array type, which is _not_ an object
- has objects (all classes are subclasses of `java.lang.Object`)

Scala's type hierarchy reflects the need to live in the JVM world.

<img src="https://i.imgur.com/hr7kB5K.png" alt="Scala Type Hierarchy" style="width: 50%"/>


### Type hierarchy highlights

- Types that map to JVM primitives are subclasses of `AnyVal`
- Types that map to JVM objects are subclasses of `AnyRef`. (You can think of `AnyRef` as an alias for `java.lang.Object`.)
- _Everything_ descends from `Any`.
- _Everything_ has `Nothing` as a (compiler-inserted) subclass. (We'll talk about `Nothing` shortly.)

In [ ]:
val a: AnyRef = ""
val b: Any = ""
//val c: AnyVal = "" // won't compile
val d: AnyVal = 10
val e: AnyVal = ()


### What is `Nothing`?

Reminder: _Everything_ is an expression in Scala.

So, what is the return type of this expression?

```scala
val res = if (2 + 2 == 4) "All is right with the world" else throw new Exception("The world is bent! Math is broken!")
```

- The type of the `if` branch is `String`
- What's the type of the `else` branch?
    - A naive assumption is that it's `Unit`.
    
But that doesn't work:

- The immediate parent class of `Unit` is `AnyVal`.
- The immediate parent class of `String` is `AnyRef`.
- The common parent is `Any`.

If the `else` clause's type were `Unit`, then the inferred type of `res` would be `Any`!

Yet that's not the case:

In [ ]:
val res1 = if (2 + 2 == 4) "All is right with the world" else () // force Unit
val res2 = if (2 + 2 == 4) "All is right with the world" else throw new Exception("The world is bent! Math is broken!")

Somehow, even though the `else` clause isn't returning a `String`, the type of the whole expression is still `String`.

Here's how that works:

- The "type" of a `throw` clause is `Nothing` (though you can't actually assign a `throw` to anything).
- `Nothing` is a subclass of `String` (because `Nothing` is a subclass of every class).
- Thus, the common parent type of `String` and `Nothing` is `String`.

`Nothing` allows type inference to work, even in the presence of a `throw` clause.

#### Other places you'll see `Nothing`

If the compiler doesn't have enough information to infer a specific type, it may substitute nothing.

Examples:

`None` is [declared as](https://www.scala-lang.org/files/archive/api/2.13.0/scala/None$.html):

```scala
object None extends Option[Nothing] with Product with Serializable
```

Ignore the `with Product with Serializable` part for now.

Declaring `None` as `Option[Nothing]` allows us to do this:

In [ ]:
val opt1: Option[String] = None
val opt2: Option[Int] = None


Similarly, `Nil` (for "empty list") is [declared as](https://www.scala-lang.org/files/archive/api/2.13.0/scala/collection/immutable/Nil$.html):

```scala
object Nil extends List[Nothing] with Product with Serializable
```

That's why we can use it with any type of list:

In [ ]:
val list1: List[String] = Nil
val list2: List[Int] = Nil
val list3 = "a" :: "b" :: "c" :: Nil

## Recap

### Packages

- Packages help organize code
- Packages can encapsulate functionality between modules
- Import statements break package encapsulation
    - One at a time
    - Many in a single statement
    - All at once
    - Can be placed anywhere in a file (scoped imports)
    - Can be used to import from packages _and_ objects

### Types

- Everything in Scala is an object
- The top type in Scala is `Any`
    - All other types are subtypes of `Any`
    - Defines functionality that all types must support
- The bottom type in Scala is `Nothing`
    - `Nothing` is a subtype of all other types
    - Provides a common way to model the beginning or end of the world